In [259]:
from lxml import etree
import pandas as pd

import tiktoken
# How to use tiktoken: https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb

encoding = tiktoken.encoding_for_model("gpt-3.5-turbo") # Encoding 'cl100k_base'

In [260]:
from pathlib import Path

udhr_dir = Path('data/udhr')
token_dfs = {}

for filename in udhr_dir.glob('*.xml'):
    language = filename.stem[5:]
    print(language, end=' ')

    with open(filename) as f:
        tree = etree.parse(f)

    text = '\n'.join([tree.find('{http://www.unicode.org/udhr}title').text] + 
        [p.text for p in tree.find('{http://www.unicode.org/udhr}preamble').findall('.//{http://www.unicode.org/udhr}para')] +
        [p.text for article in tree.findall('{http://www.unicode.org/udhr}article')
                for p in article.findall('.//{http://www.unicode.org/udhr}para')])

    token_codes = encoding.encode(text)
    df = pd.DataFrame({'decoded': [encoding.decode([token]) for token in token_codes]}).drop_duplicates()
    df['length'] = df['decoded'].str.len()
    df['byte_length'] = df['decoded'].apply(lambda d: len(d.encode('utf-8')))
    token_dfs[language] = df

spa uig_latn ind heb fra pes_1 kor tgl arb vie swe cmn_hans por_BR deu_1996 nld rus ita ukr por_PT urd tur pol eng jpn hin 

In [263]:
pd.concat([pd.DataFrame({'lang': lang, 'stat': df.byte_length.describe()}) 
           for lang, df in token_dfs.items()]).reset_index(
           ).set_index(['lang','index']).unstack()['stat'].sort_values('count', ascending=False)

index,25%,50%,75%,count,max,mean,min,std
lang,,,,,,,,
deu_1996,3.0,4.0,5.0,900.0,14.0,4.102222,1.0,1.738332
ita,3.0,4.0,5.0,841.0,12.0,4.287753,1.0,1.919314
fra,3.0,4.0,6.0,839.0,15.0,4.889154,1.0,2.466165
por_PT,3.0,4.0,6.0,826.0,15.0,4.690073,1.0,2.173855
pol,2.0,3.0,4.0,783.0,10.0,3.274585,1.0,1.366368
por_BR,3.0,4.0,6.0,770.0,14.0,4.619481,1.0,2.144794
spa,3.0,5.0,6.0,768.0,14.0,4.990885,1.0,2.282790
nld,3.0,4.0,5.0,766.0,10.0,3.781984,1.0,1.540983
tur,2.0,3.0,4.0,721.0,9.0,3.185853,1.0,1.094872


In [262]:
(lambda df:df[df['length'] > 1])(token_dfs['heb'])

,decoded,length,byte_length
7,ל,2,3
11,ב,2,3
14,�,2,4
16,ו�,2,5
32,ה,2,3
41,י�,2,5
62,א,2,3
72,מ,2,3
153,.\n,2,2
209,ש,2,3
